In [1]:
#base of the game(model)
!pip install pygame

  Using cached pygame-2.6.1-cp312-cp312-win_amd64.whl.metadata (13 kB)
Using cached pygame-2.6.1-cp312-cp312-win_amd64.whl (10.6 MB)


In [6]:
# with out Q learming 
import pygame

import random

# Initialize pygame
pygame.init()

# Colors
white = (255, 255, 255)
yellow = (255, 255, 102)
black = (0, 0, 0)
red = (213, 50, 80)
green = (0, 255, 0)
blue = (50, 153, 213)
gray = (128, 128, 128)

# Display size
dis_width = 600
dis_height = 400

# Create window
dis = pygame.display.set_mode((dis_width, dis_height))
pygame.display.set_caption('Snake Game')

# Clock
clock = pygame.time.Clock()
snake_block = 25
snake_speed = 8

font_style = pygame.font.SysFont("bahnschrift", 25)
score_font = pygame.font.SysFont("comicsansms", 35)

def Your_score(score):
    value = score_font.render("Score: " + str(score), True, yellow)
    dis.blit(value, [0, 0])

def our_snake(snake_block, snake_list):
    for x in snake_list:
        pygame.draw.rect(dis, black, [x[0], x[1], snake_block, snake_block])

def message(msg, color):
    mesg = font_style.render(msg, True, color)
    dis.blit(mesg, [dis_width / 6, dis_height / 3])

def generate_food(snake_block, dis_width, dis_height, barriers, allow_edge_chance=1/1_000_000):
    max_attempts = 1000  # Just in case — prevents infinite loops
    attempts = 0

    while attempts < max_attempts:
        # Decide if we're allowing boundary placement this time
        allow_on_edge = random.random() < allow_edge_chance

        # Generate food coordinates
        foodx = round(random.randrange(0, dis_width - snake_block) / snake_block) * snake_block
        foody = round(random.randrange(0, dis_height - snake_block) / snake_block) * snake_block

        food_rect = pygame.Rect(foodx, foody, snake_block, snake_block)

        # Check collision with barriers
        if any(food_rect.colliderect(barrier) for barrier in barriers):
            attempts += 1
            continue

        # Check if it's on boundary (within 10 pixels of edge)
        if not allow_on_edge:
            if (foodx < 10 or foodx > dis_width - snake_block - 10 or
                foody < 10 or foody > dis_height - snake_block - 10):
                attempts += 1
                continue

        return foodx, foody

    # If max attempts hit, return a safe default (center)
    return dis_width // 2, dis_height // 2



def gameLoop():
    game_over = False
    game_close = False

    x1 = dis_width / 2
    y1 = dis_height / 2
    x1_change = 0
    y1_change = 0

    snake_List = []
    Length_of_snake = 1

    # Food generation aligned to grid
    foodx = round(random.randrange(10, dis_width - snake_block - 10) / snake_block) * snake_block
    foody = round(random.randrange(10, dis_height - snake_block - 10) / snake_block) * snake_block

    # Barriers
    barrier1 = pygame.Rect(250, 250, 200, 20)
    barrier2 = pygame.Rect(400, 200, 20, 150)   # Fixed height so it doesn't go off-screen

    while not game_over:

        while game_close:
            dis.fill(blue)
            message("You lost! Press C-Play Again or Q-Quit", red)
            Your_score(Length_of_snake - 1)
            pygame.display.update()

            for event in pygame.event.get():
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_q:
                        game_over = True
                        game_close = False
                    if event.key == pygame.K_c:
                        gameLoop()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                game_over = True
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    x1_change = -snake_block
                    y1_change = 0
                elif event.key == pygame.K_RIGHT:
                    x1_change = snake_block
                    y1_change = 0
                elif event.key == pygame.K_UP:
                    y1_change = -snake_block
                    x1_change = 0
                elif event.key == pygame.K_DOWN:
                    y1_change = snake_block
                    x1_change = 0

        x1 += x1_change
        y1 += y1_change

        # Check collision with walls
        if x1 < 25 or x1 > dis_width - snake_block - 25 or y1 < 25 or y1 > dis_height - snake_block - 25:
            game_close = True

        dis.fill(blue)

        # Draw boundary (thickness = 10)
        pygame.draw.rect(dis, black, [0, 0, dis_width, dis_height], 10)

        # Draw food and barriers
        pygame.draw.rect(dis, green, [foodx, foody, snake_block, snake_block])
        pygame.draw.rect(dis, gray, barrier1)
        pygame.draw.rect(dis, gray, barrier2)

        # Snake head
        snake_Head = [x1, y1]
        snake_List.append(snake_Head)
        if len(snake_List) > Length_of_snake:
            del snake_List[0]

        # Check self collision
        for x in snake_List[:-1]:
            if x == snake_Head:
                game_close = True

        # Check if snake hits barriers
        snake_rect = pygame.Rect(x1, y1, snake_block, snake_block)
        if snake_rect.colliderect(barrier1) or snake_rect.colliderect(barrier2):
            game_close = True

        our_snake(snake_block, snake_List)
        Your_score(Length_of_snake - 1)

        pygame.display.update()

        # Food collision
        if x1 == foodx and y1 == foody:
            barrier1 = pygame.Rect(250, 250, 200, 20)
            barrier2 = pygame.Rect(400, 200, 20, 150)
            barriers = [barrier1, barrier2]  # list of barriers

            foodx, foody = generate_food(snake_block, dis_width, dis_height, barriers)

            # foodx = round(random.randrange(10, dis_width - snake_block - 10) / snake_block) * snake_block
            # foody = round(random.randrange(10, dis_height - snake_block - 10) / snake_block) * snake_block


        clock.tick(snake_speed)

    pygame.quit()

gameLoop()


In [5]:
#with Q Learning
import pygame
import random

# Initialize pygame
pygame.init()

# Colors and display
white = (255, 255, 255)
yellow = (255, 255, 102)
black = (0, 0, 0)
red = (213, 50, 80)
green = (0, 255, 0)
blue = (50, 153, 213)
gray = (128, 128, 128)

dis_width = 600
dis_height = 400
dis = pygame.display.set_mode((dis_width, dis_height))
pygame.display.set_caption('Snake AI')

clock = pygame.time.Clock()
snake_block = 25
snake_speed = 8


# Q-learning parameters
alpha = 0.1
gamma = 0.9
epsilon = 0.3  # higher exploration
Q = {}

moves = {
    "R": (snake_block, 0),
    "L": (-snake_block, 0),
    "U": (0, -snake_block),
    "D": (0, snake_block)
}

def generate_food(snake_block, dis_width, dis_height, barriers):
    while True:
        foodx = round(random.randrange(0, dis_width - snake_block) / snake_block) * snake_block
        foody = round(random.randrange(0, dis_height - snake_block) / snake_block) * snake_block
        rect = pygame.Rect(foodx, foody, snake_block, snake_block)
        if not any(rect.colliderect(b) for b in barriers):
            return foodx, foody

def reward_func(head, food, barriers, prev_distance):
    x, y = head
    if x == food[0] and y == food[1]:
        return 100
    if x < 0 or x >= dis_width or y < 0 or y >= dis_height:
        return -10
    rect = pygame.Rect(x, y, snake_block, snake_block)
    if any(rect.colliderect(b) for b in barriers):
        return -10
    current_distance = abs(x - food[0]) + abs(y - food[1])
    if current_distance < prev_distance:
        return 1
    else:
        return -0.1

def select_action(state, valid_moves):
    if random.random() < epsilon:
        return random.choice(valid_moves)
    qs = [Q.get((state, a), 0.0) for a in valid_moves]
    max_q = max(qs)
    best = [a for a, q in zip(valid_moves, qs) if q == max_q]
    return random.choice(best)

def gameLoop():
    for episode in range(1000):
        x = dis_width // 2
        y = dis_height // 2

        snake = [[x, y]]
        length = 1

        # Generate random barriers
        barriers = []
        for _ in range(2):
            bw = random.choice([100, 150, 200])
            bh = random.choice([20, 40])
            bx = random.randint(0, dis_width - bw)
            by = random.randint(0, dis_height - bh)
            barriers.append(pygame.Rect(bx, by, bw, bh))

        foodx, foody = generate_food(snake_block, dis_width, dis_height, barriers)

        done = False
        while not done:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    done = True
                    pygame.quit()
                    return

            dis.fill(blue)
            pygame.draw.rect(dis, green, [foodx, foody, snake_block, snake_block])
            for barrier in barriers:
                pygame.draw.rect(dis, gray, barrier)

            head = (x, y)
            state = (x, y, foodx, foody)

            prev_distance = abs(x - foodx) + abs(y - foody)

            valid_moves = []
            for dir, (dx, dy) in moves.items():
                nx, ny = x + dx, y + dy
                rect = pygame.Rect(nx, ny, snake_block, snake_block)
                if 0 <= nx < dis_width and 0 <= ny < dis_height and not any(rect.colliderect(b) for b in barriers):
                    valid_moves.append(dir)

            if not valid_moves:
                break

            action = select_action(state, valid_moves)
            dx, dy = moves[action]
            x += dx
            y += dy

            reward = reward_func((x, y), (foodx, foody), barriers, prev_distance)
            next_state = (x, y, foodx, foody)

            # Q-learning update
            current_q = Q.get((state, action), 0.0)
            next_qs = [Q.get((next_state, a), 0.0) for a in valid_moves]
            max_next_q = max(next_qs) if next_qs else 0.0
            Q[(state, action)] = current_q + alpha * (reward + gamma * max_next_q - current_q)

            if reward == 100:
                foodx, foody = generate_food(snake_block, dis_width, dis_height, barriers)
                # length += 1

            if reward == -10:
                done = True

            snake.append([x, y])
            if len(snake) > length:
                del snake[0]

            for s in snake:
                pygame.draw.rect(dis, black, [s[0], s[1], snake_block, snake_block])

            pygame.display.update()
            clock.tick(snake_speed)
            

        print(f"Episode {episode} finished. Length: {length}")

    pygame.quit()

gameLoop()
